In [7]:
# Main Program

#######################################   引入所需套件   #######################################
import models_function as mf
import cutword
import pandas as pd
import time
import string
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
import pymysql
from joblib import dump, load

#######################################   前置設定   #######################################

# 公開資訊觀測站網址
url2 = 'https://mops.twse.com.tw/mops/web/t05sr01_1'

# load斷切詞需要的tokenizer
my_tokenizer = load('/Users/wangyuda/Desktop/文字探勘/Program_Final_test/models_function_box/my_tokenizer.joblib')

# 判斷新增新聞筆數的起始計數
old_number = 0

#######################################   登入網站   #######################################

# 網站登入資訊，帳號與密碼
id = "p314008dsa1"
password = "MaTuItgeeu30hy$D63"

# 輸入我們的WordPress網址，若試著連上該網址，應該會出現「XML-RPC server accepts POST requests only.」才對。
url = "http://ec2-52-87-157-212.compute-1.amazonaws.com//xmlrpc.php"

# 新文章要直接發布的話，就不用改，如果要變成草稿，就改成"draft"
which = "publish"

# 建立客戶端，登入網頁
wp = Client(url, id, password)

#######################################   設定定時循環功能 #######################################

last_time = 0

# 公開資訊觀測站資料約180秒更新一次，考慮程式計算時間，將循環週期設為120秒
period = 120
while(True):
    if (time.time() - last_time) > period:
        
        #######################################   進行爬蟲  #######################################
        
        # 讀取網站資料
        page = pd.read_html(url2)[7]
        new_number = len(page)
        
        #######################################   判斷新增新聞筆數   #######################################
        
        # 計算新增資料的行數 add_num
        # 若新資料筆數增加，則讀取（新資料筆數ㄧ舊資料筆數）筆資料
        add_num = 0
        if new_number > old_number:
            add_num = new_number - old_number
            old_number = new_number
        
        # 若無新增資料，則跳入下一個迴圈，繼續等120秒
        elif new_number == old_number:
            last_time = time.time()
            continue
        
        # 若新資料筆數少於舊資料筆數，代表過24點資料歸零
        elif new_number < old_number:
            old_number = 0                   # 將舊資料數歸零
            if new_number > 0:              # 若新資料數大於零，代表有新資料產生
                add_num = new_number
            else:                                   # 若新資料數等於零，代表無新資料
                last_time = time.time()      # 則跳入下一個迴圈，繼續等120秒
                continue
                
        else:
            continue
        
        # 最新資料在最上方，為了照時間順序排列，從新增資料的行數開始往上讀
        for i in range(add_num-1 , -1, -1):
            
            #######################################   整理切詞用與儲存用dataframe   #######################################
            
            # 讀取新資料，做後續的資料處理
            # 包含「公司代號、公司簡稱、發言日期、發言時間、主旨」等所有資料，將存入資料庫
            add_new = []  
            
            # 只包含主旨，之後用來切字
            cut_new = []   
            
            # 紀錄每一筆各欄位的資料
            add_new_col = []
            for j in range(5):
                add_new_col.append(page.iloc[i, j])

            add_new.append(add_new_col)
            cut_new.append(page.iloc[i, 4])

            # 製作新聞各項資訊與切詞用的 dataframe
            add_new_df = pd.DataFrame(data = add_new, columns = ["公司代號", "公司簡稱", "發言日期", "發言時間", "主旨"])
            cut_new_df = pd.DataFrame(data = cut_new, columns = ["cut_new"])
            
            #######################################   進行切詞   #######################################
            
            # 切詞
            cut = cutword.ckiptoken(cut_new_df)
            
            # 文字向量化前處理
            cut_0 = cut.content_tokenized            
            
            # 文字向量化
            token = mf.preprocess_text(my_tokenizer, cut_0)
            
            # 保留斷詞資料
            cut = cut.iloc[0, 0]
            
            #######################################   放入模型預測   #######################################
            
            # 將input 輸入模型，給出預測
            intensity = mf.predict_intensity(token)
            big_event = mf.predict_Big_Event(token)
            small_event = mf.predict_Small_Event(token)
            up_or_down = mf.predict_Month_abnormal_returns(token) 
            
            #######################################   網頁輸出   #######################################
            
            intensity = intensity[0][0]
            
            # 若事件強度絕對值大於1，則上傳到網站
            if abs(intensity) > 1:
                
                # 設定輸出內容
                title = add_new_df.iloc[0, 2] + ' ' + add_new_df.iloc[0, 1] 
                excerpt = "事件強度：" + str(intensity) + '\n' + "預期漲跌：" +  up_or_down
                category = str(add_new_df.iloc[0, 0]) + ' ' + str(add_new_df.iloc[0, 1])
                content = excerpt + '\n' + "發言時間：" + add_new_df.iloc[0, 3] + '\n' + "大事件類別：" + big_event[0][0] + '\n'
                content = content + "小事件類別：" + small_event[0][0] + '\n' + "新聞內容：" + add_new_df.iloc[0, 4]                
                
                # 依據讀取的資料，在部落格中建立新文章
                post = WordPressPost()
                post.post_status = which
                post.excerpt = excerpt
                post.title = title
                post.content = content
                post.terms_names = {"category": [category]}

                # 發布文章
                wp.call(NewPost(post))

            #######################################   存入MySQL資料庫   #######################################
            
            # 連接資料庫
            db = pymysql.connect("127.0.0.1","root","s70399S80228","sys" )

            cursor = db.cursor()
            
            # 設定簡潔的變數名稱
            a = int(add_new_df.iloc[0,0])
            b = add_new_df.iloc[0,1]
            c = add_new_df.iloc[0,2]
            d = add_new_df.iloc[0,3]
            e = int(intensity)
            f = up_or_down
            g = big_event[0][0]
            h = small_event[0][0]
            i = add_new_df.iloc[0,4]
            j = cut

            sql = """INSERT INTO result_test(Stock_id, Company_name, Date ,Time,
               Intensity, Direction, Big_event_type, Small_event_type, News_content, News_cut)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

            try:
                # Execute the SQL command
                cursor.execute(sql, (a, b, c, d, e, f, g, h, i, j))
                # Commit your changes in the database
                db.commit()
            except:
                # Rollback in case there is any error
                db.rollback()
                
        #######################################   流程結束，等待下一次爬蟲   #######################################
        
        last_time = time.time()


KeyboardInterrupt: 

In [4]:
old_number

10